# Adaptive Cards Toolkit: 05c - Rich Media Content Templates

This notebook demonstrates how to create rich media content templates for Adaptive Cards. It continues from where 05b_toolkit_content.ipynb left off, with a focus on incorporating media elements and complex layouts. Please run 01_toolkit_common.ipynb first.

## Setup

First, we need to import the common utilities defined in the toolkit_common notebook and include any new modules.

In [ ]:
# First run the common utilities notebook
%run 01_toolkit_common.ipynb

# Ensure we have all the necessary imports
import sys
import os
from datetime import datetime

# Add the parent directory to the path so we can import the toolkit
if not os.path.abspath('..') in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

# Import necessary modules directly
from src.adaptive_cards_toolkit.core.element_factory import ElementFactory
from src.adaptive_cards_toolkit.core.layout_helper import LayoutHelper
from src.adaptive_cards_toolkit.templates.templates import TemplateFactory
from src.adaptive_cards_toolkit.core.data_connector import DataConnector
import adaptive_cards.card_types as types
from adaptive_cards.card import AdaptiveCard

## Learning Objectives

In this notebook, you'll learn how to:

1. Incorporate media elements (images, videos) into content cards
2. Create news digest and documentation templates
3. Build specialized information templates
4. Understand when to use different content template types

## Content with Media Elements

Let's create a template that can incorporate media elements (images, videos) within the content itself:

In [ ]:
def create_rich_media_content_template(title, content_blocks, author=None, date=None, actions=None):
    """Create a content template with rich media elements interspersed in content.
    
    Args:
        title (str): Content title
        content_blocks (list): List of content block dictionaries with {type, content} keys
                               where type can be 'text', 'image', or 'heading'
        author (str, optional): Content author
        date (str, optional): Publication date
        actions (list, optional): List of action dictionaries
        
    Returns:
        AdaptiveCard: Configured card with rich media content
    """
    try:
        # Initialize a card with AdaptiveCard
        card = AdaptiveCard.new().version("1.5")
        
        # Add main title
        card.add_item(ElementFactory.create_heading(title))
        
        # Add metadata if provided
        metadata = []
        if author:
            metadata.append(f"By {author}")
        if date:
            metadata.append(f"Published: {date}")
            
        if metadata:
            metadata_text = " · ".join(metadata)
            card.add_item(ElementFactory.create_text(metadata_text, is_subtle=True))
        
        # Add each content block based on its type
        for block in content_blocks:
            block_type = block.get('type', 'text')
            
            if block_type == 'text':
                # Add text content with paragraph spacing
                content = block.get('content', '')
                paragraphs = content.split('\\n\\n')
                for paragraph in paragraphs:
                    card.add_item(ElementFactory.create_text(paragraph))
                    
            elif block_type == 'image':
                # Add an image with caption if provided
                image_url = block.get('url', '')
                caption = block.get('caption', '')
                alt_text = block.get('alt_text', 'Image')
                
                if image_url:
                    # Create image container using LayoutHelper
                    image_items = []
                    
                    # Add the image
                    image_items.append(ElementFactory.create_image(
                        url=image_url,
                        alt_text=alt_text
                    ))
                    
                    # Add caption if provided
                    if caption:
                        image_items.append(ElementFactory.create_text(
                            caption, 
                            is_subtle=True,
                            weight="lighter"
                        ))
                    
                    # Create container using LayoutHelper
                    image_container = LayoutHelper.create_container(items=image_items)
                    card.add_item(image_container)
                    
            elif block_type == 'heading':
                # Add a section heading
                heading_text = block.get('content', '')
                level = block.get('level', 2)
                
                card.add_item(ElementFactory.create_heading(heading_text, level=level))
        
        # Add actions if provided
        if actions:
            for action in actions:
                if action.get('type') == 'open_url':
                    card.add_action({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card.add_action({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Finalize the card - this is the key step!
        return card.create()
    
    except Exception as e:
        print(f"Error creating rich media content template: {e}")
        return None

Let's create a rich media article using this template:

Now let's create a rich media article using this template:

In [ ]:
try:
    rich_media_article = create_rich_media_content_template(
        title="Visual Guide to Adaptive Cards",
        content_blocks=[
            {
                "type": "text",
                "content": "Adaptive Cards provide a powerful way to create rich, interactive content that works across platforms. This visual guide will walk you through the key components and design principles."
            },
            {
                "type": "heading",
                "content": "Card Structure",
                "level": 2
            },
            {
                "type": "text",
                "content": "Every Adaptive Card has a consistent structure that includes a body section for content and an optional actions section for interactive elements."
            },
            {
                "type": "image",
                "url": "https://adaptivecards.io/content/adaptive-card-50.png",
                "caption": "Basic structure of an Adaptive Card",
                "alt_text": "Diagram showing the structure of an Adaptive Card",
                "size": "medium"
            },
            {
                "type": "heading",
                "content": "Key Components",
                "level": 2
            },
            {
                "type": "text",
                "content": "Adaptive Cards include various components like text blocks, images, containers, and input fields that can be combined to create rich experiences."
            },
            {
                "type": "image",
                "url": "https://adaptivecards.io/content/adaptive-card-50.png",
                "caption": "Common components in Adaptive Cards",
                "alt_text": "Diagram showing common components in Adaptive Cards",
                "size": "medium"
            },
            {
                "type": "heading",
                "content": "Design Best Practices",
                "level": 2
            },
            {
                "type": "text",
                "content": "When designing Adaptive Cards, follow these best practices:\n\n1. Keep it simple and focused\n2. Use consistent styling\n3. Optimize for mobile viewing\n4. Include clear calls to action\n5. Test across platforms"
            }
        ],
        author="Design Team",
        date="February 24, 2023",
        actions=[
            {
                "type": "open_url",
                "title": "View Designer",
                "url": "https://adaptivecards.io/designer/"
            },
            {
                "type": "submit",
                "title": "Download Guide",
                "data": {"action": "download_guide"}
            }
        ]
    )
    
    display_card(rich_media_article, "Rich Media Article Card")
    
except Exception as e:
    print(f"Error creating rich media article: {e}")

The rich media content template allows us to create sophisticated articles with various content types mixed together. This approach gives you much more flexibility than a traditional article template.

## Content with Metadata

Now let's create a content template that prominently displays author information and other metadata:

## Next Steps

In the next notebook, 05d_toolkit_profiles.ipynb, we'll explore:
1. Creating profile cards with contact information
2. Building content with rich metadata displays
3. Implementing a dynamic content rendering system
4. Creating more specialized templates

Be sure to check out 05e_toolkit_specialized.ipynb for documentation templates and news digests after completing this series.

In [ ]:
def create_content_with_metadata_template(title, content, metadata=None, image_url=None, actions=None):
    """Create a content template with rich metadata display.
    
    Args:
        title (str): Content title
        content (str): Main content text
        metadata (dict, optional): Dictionary of metadata (author, date, categories, etc.)
        image_url (str, optional): URL to a header image
        actions (list, optional): List of action dictionaries
        
    Returns:
        AdaptiveCard: Configured card with metadata
    """
    try:
        # Initialize a card with AdaptiveCard
        card = AdaptiveCard.new().version("1.5")
        
        # Add header image if provided
        if image_url:
            card.add_item(ElementFactory.create_image(
                url=image_url,
                alt_text=f"Image for {title}"
            ))
        
        # Add title
        card.add_item(ElementFactory.create_heading(title))
        
        # Add metadata in a structured format if provided
        if metadata and isinstance(metadata, dict):
            # Create metadata container
            metadata_items = []
            metadata_container = LayoutHelper.create_container(
                items=metadata_items, 
                style="emphasis"
            )
            
            # Author with avatar if available
            if 'author' in metadata or 'author_avatar' in metadata:
                author_items = []
                author_container = LayoutHelper.create_container(
                    items=author_items,
                    spacing="small"
                )
                
                # Create column layout for author info
                columns = []
                
                # Avatar column
                if 'author_avatar' in metadata:
                    avatar_items = [
                        ElementFactory.create_image(
                            url=metadata['author_avatar'],
                            style="person",
                            alt_text=f"Avatar for {metadata.get('author', 'author')}"
                        )
                    ]
                    avatar_column = LayoutHelper.create_column(width="auto", items=avatar_items)
                    columns.append(avatar_column)
                
                # Author info column
                author_info_items = []
                
                if 'author' in metadata:
                    author_info_items.append(ElementFactory.create_text(
                        metadata['author'],
                        weight="bolder"
                    ))
                
                # Author title/role if available
                if 'author_title' in metadata:
                    author_info_items.append(ElementFactory.create_text(
                        metadata['author_title'],
                        is_subtle=True
                    ))
                
                author_info_column = LayoutHelper.create_column(width="stretch", items=author_info_items)
                columns.append(author_info_column)
                
                # Create column set and add to author container
                column_set = LayoutHelper.create_column_set(columns)
                metadata_items.append(column_set)
            
            # Publication details
            pub_details = []
            
            if 'date' in metadata:
                pub_details.append(f"Published: {metadata['date']}")
                
            if 'reading_time' in metadata:
                pub_details.append(f"{metadata['reading_time']} min read")
                
            if 'categories' in metadata and metadata['categories']:
                if isinstance(metadata['categories'], list):
                    categories = ', '.join(metadata['categories'])
                    pub_details.append(f"Categories: {categories}")
                else:
                    pub_details.append(f"Category: {metadata['categories']}")
            
            if pub_details:
                pub_details_text = ElementFactory.create_text(
                    ' · '.join(pub_details),
                    is_subtle=True
                )
                metadata_items.append(pub_details_text)
            
            card.add_item(metadata_container)
        
        # Add content with paragraph spacing
        paragraphs = content.split('\\n\\n')
        for paragraph in paragraphs:
            text_block = ElementFactory.create_text(paragraph)
            card.add_item(text_block)
        
        # Add actions if provided
        if actions:
            for action in actions:
                if action.get('type') == 'open_url':
                    card.add_action({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card.add_action({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Finalize the card - this is the key step!
        return card.create()
    
    except Exception as e:
        print(f"Error creating content with metadata template: {e}")
        return None

Let's create a content article with rich metadata:

In [ ]:
try:
    metadata_article = create_content_with_metadata_template(
        title="Best Practices for AI-Generated Content Cards",
        content="When using AI to generate adaptive cards, it's important to follow certain best practices to ensure the content is accessible, relevant, and engaging.\n\nFirst, ensure that all generated cards include proper alt text for images and follow accessibility guidelines. AI systems should be trained to provide these automatically.\n\nSecond, personalize content based on user context whenever possible. This creates a more engaging experience and makes the information more relevant to the individual user.\n\nFinally, implement a feedback loop to improve card generation over time. User interactions with cards can provide valuable data on what works and what doesn't.",
        metadata={
            "author": "Dr. Maya Rodriguez",
            "author_title": "AI Communications Lead",
            "author_avatar": "https://adaptivecards.io/content/adaptive-card-50.png",
            "date": "February 23, 2023",
            "reading_time": "4",
            "categories": ["AI", "Design", "Best Practices"]
        },
        image_url="https://adaptivecards.io/content/adaptive-card-50.png",
        actions=[
            {
                "type": "open_url",
                "title": "View Full Article",
                "url": "https://example.com/articles/ai-content-cards"
            },
            {
                "type": "submit",
                "title": "Save for Later",
                "data": {"action": "save_article", "id": "ai-content-cards"}
            }
        ]
    )
    
    display_card(metadata_article, "Article with Rich Metadata")
    
except Exception as e:
    print(f"Error creating metadata article: {e}")

## Dynamic Content Rendering

Now let's create a more advanced system that can dynamically render different types of content based on the data provided:

In [ ]:
class ContentTemplateSystem:
    """A system for dynamically rendering different types of content."""
    
    def __init__(self):
        """Initialize the content template system."""
        pass
    
    def render_content(self, content_data):
        """Render a content card based on the type and data provided.
        
        Args:
            content_data (dict): Data dictionary containing content information
                                 Must include 'type' and other required fields
        
        Returns:
            AdaptiveCard: Rendered card based on content type
        """
        try:
            content_type = content_data.get('type', 'basic')
            
            if content_type == 'article':
                return self._render_article(content_data)
            elif content_type == 'sectioned':
                return self._render_sectioned_content(content_data)
            elif content_type == 'rich_media':
                return self._render_rich_media(content_data)
            elif content_type == 'profile':
                return self._render_profile(content_data)
            else:  # Default to basic
                return self._render_basic(content_data)
        
        except Exception as e:
            print(f"Error rendering content: {e}")
            return None
    
    def _render_basic(self, data):
        """Render a basic content card."""
        # Import from previous notebook
        from src.adaptive_cards_toolkit.core.card_builder import create_article_template 
        return create_article_template(
            title=data.get('title', 'Untitled'),
            content=data.get('content', ''),
            image_url=data.get('image_url'),
            author=data.get('author'),
            date=data.get('date'),
            actions=data.get('actions')
        )
    
    def _render_article(self, data):
        """Render an article content card."""
        # Import from previous notebook
        from src.adaptive_cards_toolkit.core.card_builder import create_article_template
        return create_article_template(
            title=data.get('title', 'Untitled'),
            content=data.get('content', ''),
            image_url=data.get('image_url'),
            author=data.get('author'),
            date=data.get('date'),
            actions=data.get('actions')
        )
    
    def _render_sectioned_content(self, data):
        """Render a sectioned content card."""
        # Import from previous notebook
        from src.adaptive_cards_toolkit.core.card_builder import create_sectioned_content_template
        return create_sectioned_content_template(
            title=data.get('title', 'Untitled'),
            sections=data.get('sections', []),
            image_url=data.get('image_url'),
            author=data.get('author'),
            date=data.get('date'),
            actions=data.get('actions')
        )
    
    def _render_rich_media(self, data):
        """Render a rich media content card."""
        return create_rich_media_content_template(
            title=data.get('title', 'Untitled'),
            content_blocks=data.get('content_blocks', []),
            author=data.get('author'),
            date=data.get('date'),
            actions=data.get('actions')
        )
    
    def _render_profile(self, data):
        """Render a profile content card."""
        try:
            # Initialize a card with AdaptiveCard
            card = AdaptiveCard.new().version("1.5")
            
            # Add profile header
            header_items = []
            header = LayoutHelper.create_container(
                items=header_items,
                style="emphasis"
            )
            
            # Create columns for profile layout
            columns = []
            
            # Profile image column
            if 'profile_image' in data:
                image_items = [
                    ElementFactory.create_image(
                        url=data['profile_image'],
                        style="person",
                        alt_text=f"Profile image for {data.get('name', 'User')}"
                    )
                ]
                image_column = LayoutHelper.create_column(width="auto", items=image_items)
                columns.append(image_column)
            
            # Profile details column
            details_items = []
            
            if 'name' in data:
                details_items.append(ElementFactory.create_heading(
                    data['name'],
                    level=2
                ))
            
            if 'title' in data:
                details_items.append(ElementFactory.create_text(
                    data['title'],
                    weight="bolder"
                ))
            
            if 'organization' in data:
                details_items.append(ElementFactory.create_text(
                    data['organization']
                ))
            
            details_column = LayoutHelper.create_column(width="stretch", items=details_items)
            columns.append(details_column)
            
            # Create column set and add to header
            column_set = LayoutHelper.create_column_set(columns)
            header_items.append(column_set)
            
            card.add_item(header)
            
            # Add contact information using a fact set
            if any(key in data for key in ['email', 'phone', 'location', 'website']):
                contact_facts = []
                
                if 'email' in data:
                    contact_facts.append({"title": "Email", "value": data['email']})
                    
                if 'phone' in data:
                    contact_facts.append({"title": "Phone", "value": data['phone']})
                    
                if 'location' in data:
                    contact_facts.append({"title": "Location", "value": data['location']})
                    
                if 'website' in data:
                    contact_facts.append({"title": "Website", "value": data['website']})
                
                # Add facts as individual text items in a container
                contact_items = []
                for fact in contact_facts:
                    contact_items.append(ElementFactory.create_text(
                        f"{fact['title']}: {fact['value']}"
                    ))
                
                contact_container = LayoutHelper.create_container(
                    items=contact_items,
                    spacing="medium"
                )
                card.add_item(contact_container)
            
            # Add bio/about section
            if 'bio' in data:
                bio_items = [
                    ElementFactory.create_heading("About", level=3),
                    ElementFactory.create_text(data['bio'])
                ]
                
                bio_container = LayoutHelper.create_container(
                    items=bio_items,
                    spacing="medium"
                )
                card.add_item(bio_container)
            
            # Add skills/expertise
            if 'skills' in data and data['skills']:
                if isinstance(data['skills'], list):
                    skills_text = ", ".join(data['skills'])
                else:
                    skills_text = data['skills']
                
                skills_items = [
                    ElementFactory.create_heading("Skills & Expertise", level=3),
                    ElementFactory.create_text(skills_text)
                ]
                
                skills_container = LayoutHelper.create_container(
                    items=skills_items,
                    spacing="medium"
                )
                card.add_item(skills_container)
            
            # Add actions
            if 'actions' in data and data['actions']:
                for action in data['actions']:
                    if action.get('type') == 'open_url':
                        card.add_action({
                            "type": "Action.OpenUrl",
                            "title": action.get('title', 'Open'),
                            "url": action.get('url', '#')
                        })
                    elif action.get('type') == 'submit':
                        card.add_action({
                            "type": "Action.Submit",
                            "title": action.get('title', 'Submit'),
                            "data": action.get('data', {})
                        })
            
            # Finalize the card - this is the key step!
            return card.create()
        
        except Exception as e:
            print(f"Error rendering profile: {e}")
            return None

Let's try our dynamic content rendering system with different content types:

In [ ]:
try:
    # Initialize the content template system
    content_system = ContentTemplateSystem()
    
    # Define a profile content
    profile_data = {
        "type": "profile",
        "name": "Dr. Emma Chen",
        "title": "Lead AI Researcher",
        "organization": "Adaptive Intelligence Labs",
        "profile_image": "https://adaptivecards.io/content/adaptive-card-50.png",
        "email": "emma.chen@example.com",
        "phone": "+1 (555) 123-4567",
        "location": "San Francisco, CA",
        "website": "www.example.com/echen",
        "bio": "Dr. Emma Chen specializes in adaptive AI systems that can generate rich, interactive content for user interfaces. Her research focuses on making AI-generated content more accessible, engaging, and contextually relevant.",
        "skills": ["AI Research", "Machine Learning", "Natural Language Processing", "Human-Computer Interaction", "Adaptive Interfaces"],
        "actions": [
            {
                "type": "open_url",
                "title": "View Publications",
                "url": "https://example.com/publications/echen"
            },
            {
                "type": "submit",
                "title": "Contact",
                "data": {"action": "contact", "id": "echen"}
            }
        ]
    }
    
    # Render the profile content
    profile_card = content_system.render_content(profile_data)
    display_card(profile_card, "Dynamic Profile Card")
    
except Exception as e:
    print(f"Error creating dynamic content: {e}")

## Specialized Content Templates

Let's create a few specialized content templates for specific uses.

In [ ]:
def create_documentation_page_template(title, content, metadata=None, code_samples=None, actions=None):
    """Create a documentation page template."""
    try:
        # Initialize card with AdaptiveCard
        card = AdaptiveCard.new().version("1.5")
        
        # Add title
        card.add_item(ElementFactory.create_heading(title))
        
        # Add metadata if provided
        if metadata:
            metadata_text = []
            
            if 'version' in metadata:
                metadata_text.append(f"Version: {metadata['version']}")
                
            if 'updated' in metadata:
                metadata_text.append(f"Updated: {metadata['updated']}")
                
            if 'category' in metadata:
                metadata_text.append(f"Category: {metadata['category']}")
                
            if metadata_text:
                card.add_item(ElementFactory.create_text(
                    " | ".join(metadata_text),
                    is_subtle=True
                ))
        
        # Add content
        paragraphs = content.split('\n\n')
        for paragraph in paragraphs:
            card.add_item(ElementFactory.create_text(paragraph))
        
        # Add code samples if provided
        if code_samples and isinstance(code_samples, list):
            for sample in code_samples:
                # Create a container for the code sample
                code_items = []
                
                if 'language' in sample:
                    code_items.append(ElementFactory.create_text(
                        f"Language: {sample['language']}",
                        is_subtle=True
                    ))
                
                if 'title' in sample:
                    code_items.append(ElementFactory.create_text(
                        sample['title'],
                        weight="bolder"
                    ))
                
                if 'code' in sample:
                    code_items.append(ElementFactory.create_text(
                        sample['code'],
                        is_monospace=True,
                        wrap=True
                    ))
                
                code_container = LayoutHelper.create_container(
                    items=code_items,
                    style="emphasis",
                    spacing="medium"
                )
                
                card.add_item(code_container)
        
        # Add actions if provided
        if actions:
            for action in actions:
                if action.get('type') == 'open_url':
                    card.add_action({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card.add_action({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Finalize the card
        return card.create()
    
    except Exception as e:
        print(f"Error creating documentation page: {e}")
        return None

In [ ]:
def create_news_digest_template(title, news_items, date=None, actions=None):
    """Create a news digest template with multiple news items."""
    try:
        # Initialize card with AdaptiveCard
        card = AdaptiveCard.new().version("1.5")
        
        # Add title
        card.add_item(ElementFactory.create_heading(title))
        
        # Add date if provided
        if date:
            card.add_item(ElementFactory.create_text(f"Published: {date}", is_subtle=True))
        
        # Add each news item
        for i, item in enumerate(news_items):
            # Add separator between items (except before the first item)
            if i > 0:
                # Use a dictionary-based separator
                card.add_item({"type": "Separator"})
            
            # Create container for the news item
            item_items = []
            
            # Create columns for layout
            column_items = []
            
            # Left column for content
            left_items = []
            
            # Add news item title
            if 'title' in item:
                left_items.append(ElementFactory.create_text(
                    item['title'],
                    weight="bolder"
                ))
            
            # Add news item summary
            if 'summary' in item:
                left_items.append(ElementFactory.create_text(
                    item['summary'],
                    wrap=True
                ))
            
            # Add source and date
            source_info = []
            if 'source' in item:
                source_info.append(item['source'])
            if 'date' in item:
                source_info.append(item['date'])
            
            if source_info:
                left_items.append(ElementFactory.create_text(
                    " | ".join(source_info),
                    is_subtle=True
                ))
            
            left_column = LayoutHelper.create_column(width="stretch", items=left_items)
            column_items.append(left_column)
            
            # Right column for image (if provided)
            if 'image_url' in item:
                right_items = [
                    ElementFactory.create_image(
                        url=item['image_url'],
                        alt_text=f"Image for {item.get('title', 'news item')}"
                    )
                ]
                
                right_column = LayoutHelper.create_column(width="auto", items=right_items)
                column_items.append(right_column)
            
            # Create column set
            column_set = LayoutHelper.create_column_set(column_items)
            item_items.append(column_set)
            
            # Add "Read more" link if URL provided
            if 'url' in item:
                item_items.append(ElementFactory.create_text(
                    "[Read more](" + item['url'] + ")"
                ))
            
            # Create container for the item
            item_container = LayoutHelper.create_container(
                items=item_items,
                spacing="medium"
            )
            
            card.add_item(item_container)
        
        # Add actions if provided
        if actions:
            for action in actions:
                if action.get('type') == 'open_url':
                    card.add_action({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card.add_action({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Finalize the card
        return card.create()
    
    except Exception as e:
        print(f"Error creating news digest: {e}")
        return None

Let's create some examples using these templates:

## Key Takeaways

In this notebook, you've learned:

1. How to incorporate media elements like images into your content cards
2. How to create rich media templates with mixed content types
3. How to structure complex content with headings, text, and images
4. How to add captions and alternative text for accessibility

In the next notebook, 05d_toolkit_profiles.ipynb, we'll explore profile cards and metadata-rich content templates.

## Exercises

1. Create a rich media article about a product with images, headings, and descriptive text.
2. Modify the rich media template to support video embedding through a URL.
3. Create a photo gallery template that displays multiple images with captions.
4. Extend the rich media content template to support different text formatting options (bold, italic, etc.).
5. Create a step-by-step guide template with numbered steps and accompanying images.

In [ ]:
class ContentTemplateLibrary:
    """A comprehensive library of content templates."""
    
    def __init__(self):
        """Initialize the content template library."""
        self.template_system = ContentTemplateSystem()
    
    def create_article(self, title, content, image_url=None, author=None, date=None, actions=None):
        """Create a basic article template."""
        # Import from previous notebook
        from src.adaptive_cards_toolkit.core.card_builder import create_article_template
        return create_article_template(
            title=title,
            content=content,
            image_url=image_url,
            author=author,
            date=date,
            actions=actions
        )
    
    def create_sectioned_article(self, title, sections, image_url=None, author=None, date=None, actions=None):
        """Create a sectioned article template."""
        # Import from previous notebook
        from src.adaptive_cards_toolkit.core.card_builder import create_sectioned_content_template
        return create_sectioned_content_template(
            title=title,
            sections=sections,
            image_url=image_url,
            author=author,
            date=date,
            actions=actions
        )
    
    def create_rich_media_article(self, title, content_blocks, author=None, date=None, actions=None):
        """Create a rich media article template."""
        return create_rich_media_content_template(
            title=title,
            content_blocks=content_blocks,
            author=author,
            date=date,
            actions=actions
        )
    
    def create_article_with_metadata(self, title, content, metadata=None, image_url=None, actions=None):
        """Create an article with rich metadata."""
        return create_content_with_metadata_template(
            title=title,
            content=content,
            metadata=metadata,
            image_url=image_url,
            actions=actions
        )
    
    def create_profile(self, profile_data):
        """Create a profile template."""
        profile_data["type"] = "profile"
        return self.template_system.render_content(profile_data)
    
    def create_documentation_page(self, title, content, metadata=None, code_samples=None, actions=None):
        """Create a documentation page template."""
        return create_documentation_page_template(
            title=title,
            content=content,
            metadata=metadata,
            code_samples=code_samples,
            actions=actions
        )
    
    def create_news_digest(self, title, news_items, date=None, actions=None):
        """Create a news digest template with multiple news items."""
        return create_news_digest_template(
            title=title,
            news_items=news_items,
            date=date,
            actions=actions
        )
    
    def create_from_json(self, template_data):
        """Create a content template from JSON data."""
        return self.template_system.render_content(template_data)

## Key Takeaways

In this notebook, you've learned:

1. How to incorporate media elements like images into your content cards
2. How to add rich metadata displays for articles and content
3. How to build a dynamic content rendering system that adapts to different content types
4. How to create specialized content templates for documentation and news digests
5. How to organize templates into a comprehensive library

In the next notebook, we'll explore form templates for collecting user input and feedback.

## Exercises

1. Create a product spotlight template that showcases a product with images, features, pricing, and reviews.
2. Create a tutorial template that includes numbered steps, code examples, and screenshots.
3. Create a team member directory template that displays multiple profile cards in a grid layout.
4. Create a portfolio template that showcases projects with images, descriptions, and links.
5. Modify the news digest template to support categorized news items with color-coded category labels.